# Mixed-integer programming

Operations research often involves models where you need discrete decisions. 

Suppose Iain wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}

In particular, let us work through the following example:

\begin{align*}
    \max\:& x_1 + x_2 \\
    \text{s.t.}\:& x_1 + 2x_2 \leq 1.5 \\
    & x_1, x_2 \in \{0,1\}
\end{align*}

**Discussion**: Suppose you have a solver that can only solve linear optimization problems. How might you try to solve it?

## Branch-and-Bound
One way is to enumerate each possible value for the decision variables $x_1=x, x_2=y$ and compare the cost. Let's visualize this approach as a search tree:

![Full Tree](http://www.mit.edu/~huchette/.tmp/fulltree.svg)

As we go down the arcs of the tree we restrict our problem more and more, we must have that:

>If node ``q`` is descended from node ``p``, we must have that the optimal cost of subproblem ``q`` is no more than that for node ``p``

This leads us to a powerful tool in solving these enumeration problems: 

>If I can show you that the optimal cost for subproblem ``q`` is _less_ than the optimal cost for the original problem, the same is true for any descendent of ``q``. 

That is, we can _prune_ the tree and safely discard some nodes, kind of like this:
![Pruning](http://www.mit.edu/~huchette/.tmp/fathom.svg)

**Discussion**: What can we do if we have integer (and not just binary) variables?

## Worked Example
First, we solve the LP relaxation and get $(x_1^*,x_2^*) = (1,0.25)$.

![fig1](http://www.mit.edu/~huchette/.tmp/fig2.svg)

This isn't integer feasible, so we consider two cases for $x_2$: (i) when $x_2=0$ and (ii) when $x_2=1$. The subproblem with $x_2 = 1$ is infeasible, and the subproblem with $x_2 = 0$ is feasible with solution $(x_1^*,x_2^*) = (1,0)$. This is integer feasible, so we update our lower bound. We've also exhausted the tree, so we have our optimal solution!

## Implementation in JuMP

Let's solve our simple knapsack problem for Iain and see what the solver spits out.

In [1]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@variable(m, x, Bin) # or Int
@variable(m, y, Bin)

@constraint(m, x + 2y ≤ 1.5)
@objective(m, Max, x + y)

solve(m)

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 1
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 1 
Pool objective bound 1

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


:Optimal

This is kind of dull since Gurobi solves this before we ever get to the branch-and-bound tree! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=100$, naïve enumeration would create $2^{100}$ nodes, which would take quite some time. How does the solver actually tackle it?

In [6]:
?rand

search: rand randn rand! randn! randexp randperm randjump randexp! randcycle



```
rand([rng], [S], [dims...])
```

Pick a random element or array of random elements from the set of values specified by `S`; `S` can be

  * an indexable collection (for example `1:n` or `['x','y','z']`), or
  * a type: the set of values to pick from is then equivalent to `typemin(S):typemax(S)` for integers (this is not applicable to `BigInt`), and to $[0, 1)$ for floating point numbers;

`S` defaults to `Float64`.


In [2]:
srand(100)
N = 100

m = Model(solver=GurobiSolver())
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)

Optimize a model with 10 rows, 100 columns and 1000 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 2.35937
Presolve time: 0.00s
Presolved: 10 rows, 100 columns, 1000 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 4.014179e+00, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
H    0     0                       3.3780464    4.01418  18.8%     -    0s
     0     0    3.97490    0    9    3.37805    3.97490  17.7%     -    0s
     0     0    3.97490    0    7    3.37805    3.97490  17.7%     -    0s
     0     0    3.97490    0    9  

:Optimal

## Understanding solver output

First, it solves the LP relaxation and reports back:
```
Root relaxation: objective 4.014179e+00, 18 iterations, 0.00 seconds
```
Now it explores the branch-and-bound tree, and updates us as it goes along. Let's look at just the first line:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
```
We see that the information is broken down into four main columns:

1. ``Nodes``: Global node information
    * how many nodes have we looked at
    * how many do we have in our queue
2. ``Current Node``
    * objective
    * depth in the tree
    * number of noninteger variables in the solution
3. ``Objective Bounds``
    * Best incumbent (lower bound)
    * node upper bound
    * the gap between the two
4. ``Work``
    * average simplex iterations per node
    * total elapsed time

Finally, we get a neat summary of the cutting planes Gurobi found useful:
```
Cutting planes:
  Gomory: 2
  Cover: 6
  MIR: 9
```
All told, we explored 243  nodes, much less than the $2^{100}$ we were worried about. All this only took 851 simplex iterations and 0.1 seconds.

Now what about those ``H``s that appear? That tells us that Gurobi ran a heuristic and found a new best solution. You can see for yourself, as the incumbent value increases while the bound remains the same:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
H    0     0                       3.3780464    4.01418  18.8%     -    0s
```
You'll also sometimes see a ``*`` instead of the ``H``, which says that the feasible solution came from branching instead of heuristics.

## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

The most natural formulation of this problem would probably be something like

$$x_{i,j} \in \{1, 2, \dots, 9\}$$

which is of course something we can do with integer programming:

$$1 \leq x_{i,j} \leq 9, ~ integer$$

The challenge now is the constraints. One observation is that the numbers 1 to 9 add up to 45, so we could try something like:

$$ \sum_{j=1}^9 x_{i,j} = 45 $$

We could probably make this work, but we'd need many more constraints. Instead, lets change our **variables**: $x_{i,j,k} = 1$ iff the number $k$ will appear in cell $(i,j)$. We can now use our 0-1 integer programming knowledge to model the problem. Consider the "row" constraints: we want each number to appear once and only once. This is equivalent to saying that

$$\sum_{j=1}^9 x_{i,j,k} = 1 \quad \forall i, k$$

We can now model this as a $9\times 9\times 9$ dimensional problem - thats a lot of binary variables, surely Gurobi won't like that!

In [3]:
init_vals = [
3 1 0 0 5 8 0 0 4;
0 0 9 3 2 0 0 0 0;
0 2 5 1 0 4 0 9 0;
0 0 0 0 0 0 3 8 9;
0 0 8 0 0 0 5 0 0;
5 4 6 0 0 0 0 0 0;
0 8 0 2 0 3 6 5 0;
0 0 0 0 7 1 4 0 0;
7 0 0 4 8 0 0 2 1]

function sudoku2()
    sudoku = Model(solver=GurobiSolver())

    @variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

    @constraints sudoku begin
        # Constraint 1 - Only one value appears in each cell
        # Constraint 2 - Each value appears in each row once only
        # Constraint 3 - Each value appears in each column once only
        cell[i=1:9, j=1:9], sum(x[i,j,:]) == 1
         row[i=1:9, k=1:9], sum(x[i,:,k]) == 1
         col[j=1:9, k=1:9], sum(x[:,j,k]) == 1
        # Constraint 4 - Each value appears in each 3x3 subgrid once only
        subgrid[i=1:3:7,j=1:3:7,val=1:9], sum(x[i:i+2,j:j+2,val]) == 1
    end

    # Initial solution
    for row in 1:9, col in 1:9
        if init_vals[row,col] != 0
            @constraint(sudoku, x[row, col, init_vals[row, col]] == 1)
        end
    end
    
    solve(sudoku)
    
    getvalue(x)
end
@time soln = sudoku2();

Optimize a model with 358 rows, 729 columns and 2950 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 358 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap -
  0.737858 seconds (530.36 k allocations: 21.626 MB, 1.14% gc time)


## Presolving the Problem
Can you see the lines
```
Optimize a model with 358 rows, 729 columns and 2950 nonzeros
Presolve removed 358 rows and 729 columns
```
? Gurobi was able to use logic to deduce the value of every variable - no linear relaxation required! This "magic" is actually how a human might solve it. Consider the following:

We know that $x_{2,6,5}$ is fixed at 1 because it is one of the provided values. So we can actually replace $x_{2,6,5}$ wherever it appears in the constraints with the constant 1:

"The value 5 must appear in row 2":
$$x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,6,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + 1 + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 0$$

"The value 5 must appear in column 6":
$$x_{1,6,5} + x_{2,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + 1 + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 0$$

and so on. Because all those other variables can only be 0 or 1, and their sum is 0, they must all be 0. Thus Gurobi presolve can perform the following procedure:
1. Replace all the fixed values with constants
2. Use constraints to fix variables, e.g. at 0 (or 1)
3. Go to 1 unless step 2 did nothing.

A small problem arises when there a multiple solutions to the problem - a random selection has to be made. Gurobi will treat this case as "trying to find a feasible solution" - it will fix a variable, and follow the implications through.

Gurobi can do similar presolving implications for continuous decisions too, e.g.
$$x \in \{ 0, 1 \}$$
$$x \leq \frac{1}{2}$$

will presolve to $x = 0$: Gurobi knows that all the variables are integer, so we can safe round down the right-hand-side to the closest integer.

>**\[Exercise\]**: Pre-solve

> What if the right-hand-side is more complicated? What can you do to "tighten" the following constraint and the variables in it?

> $$ 6x_1 + 10x_2 + 12x_3 + 19x_4 \leq 15$$

> Hints:
> * Can you find common factors in the coefficients?
> * Can you round anything?
> * Can you prove things about the variable values?

> (all variables are binary)

>**\[Solution\]**: Pre-solve

> * $6x_1 + 10x_2 + 12x_3 + 19x_4 \leq 15$
> * $x_4$ must be 0, because if $x_4$ was 1 then the constaint would be violated
> * $6x_1 + 10x_2 + 12x_3 \leq 15$
> * All coefficients on left are multiples of 2, so divide through by 2
> * $3x_1 + 5x_2 + 6x_3 \leq 7.5$
> * All coefficients are integer, all variables are binary, so can round down the RHS.
> * $3x_1 + 5x_2 + 6x_3 \leq 7$
> * We note though that only one of these variables can be one at a time, or the constraint would be violated, so the best we can do is actually
> * $x_1 + x_2 + x_3 \leq 1$